In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import datetime
from influxdb import InfluxDBClient
databasename = ['MG1']
client = InfluxDBClient('120.107.146.56', 8086, 'ncue01', 'Q!A@Z#WSX', 'MG1') 

In [2]:
## 在線使用設置##############
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [3]:
#轉為小時單位
def bulid_hour_data(data_raw):
    data_raw['TIME_TO_INTERVAL'] = pd.to_datetime(data_raw['TIME_TO_INTERVAL'])
    data_raw['Hour'] = data_raw['TIME_TO_INTERVAL'].dt.hour
    data_raw['Date'] = data_raw['TIME_TO_INTERVAL'].dt.date
    data_raw = data_raw.groupby(['Date', 'Hour']).mean().reset_index()
    data_raw['TIME_TO_INTERVAL'] = data_raw.apply(lambda raw:'{} {:02d}:00:00'.format(raw['Date'], int(raw['Hour'])), axis=1)
    return data_raw
#轉為15分鐘單位
def build_quarter_hour_data(data_raw):
    data_raw['TIME_TO_INTERVAL'] = pd.to_datetime(data_raw['TIME_TO_INTERVAL'])
    data_raw['Quarter'] = data_raw['TIME_TO_INTERVAL'].dt.floor('15T')
    data_raw['Date'] = data_raw['Quarter'].dt.date
    data_raw = data_raw.groupby(['Date', 'Quarter']).mean().reset_index()
    data_raw['TIME_TO_INTERVAL'] = data_raw['Quarter'].apply(lambda dt: dt.strftime('%Y-%m-%d %H:%M:%S'))
    return data_raw

In [4]:
#抓取資料庫全部資料
def get_power():
    tablename = 'MG1_PV'
    result = client.query(f'SELECT * FROM {tablename} where Time >= 2021-05-03') 
#     result = client.query(f'SELECT * FROM {tablename}') 
    data =  list(result.get_points())
    data = pd.DataFrame(data)

    data = data.rename(columns={'Time':'TIME_TO_INTERVAL'})
    data = build_quarter_hour_data(data)
    data.to_csv('Dataset/merge_power_data(hour).csv')

In [5]:
#根據時間抓取該時資料
def get_power_date(date):
    tablename = 'MG1_PV'
    result = client.query(f"SELECT * FROM {tablename} where Time >= '{date}' - 8h") 
#     result = client.query(f"SELECT * FROM {tablename} where Time >= '{date}'") 
#     result = client.query(f'SELECT * FROM {tablename}') 
    data =  list(result.get_points())
    data = pd.DataFrame(data)
    data = data.rename(columns={'Time':'TIME_TO_INTERVAL'})
    data = build_quarter_hour_data(data)
    data = data[['kP','TIME_TO_INTERVAL']]
    data.set_axis(['Power','TIME_TO_INTERVAL'], axis='columns', inplace=True)
    return data
#     data.to_csv('Dataset/merge_power_data(hour).csv')

In [6]:
def new_power(row, datas):
    data = datas[datas['TIME_TO_INTERVAL'].eq(row['TIME_TO_INTERVAL'])]
    if data.any(axis=None):
        return data['kP'].values[0]
    else:
        return row['Power']

# merge_raw['Power'] = merge_raw.apply(lambda x:new_power(x, data), axis=1)

# 發送email

In [7]:
import smtplib
import schedule
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

def send_email():
    from_email = 'idsl5397@gmail.com'
    to_email = 'idsl5397@gmail.com'
    subject = 'Scheduled Email'
    body = 'get_power_15正常執行'

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)  # Update with your SMTP server details
    server.starttls()
    server.login(from_email, 'loqcvdrhhmgdlijl')  # Update with your email password

    server.sendmail(from_email, to_email, msg.as_string())
    server.quit()

# 每小時02分爬取一次

In [ ]:
while(True):
    localtime = time.localtime()
    result = time.strftime("%M:%S", localtime)
    result_hour = time.strftime("%H", localtime)
      
  
    if(result<='02:00'):
        if (result_hour == '12'):
            send_email()
            print("EMAIL SEND OK")
        
        start_time = time.time()
        # 設定要產生的開始與結束日期
        day = datetime.datetime.today().strftime('%Y-%m-%d')
        day = pd.to_datetime(day)
        start = pd.to_datetime(str(day.year)+'-'+str(day.month)+'-'+str(day.day))-datetime.timedelta(days=3)
        print(start)
        data = get_power_date(start)
        old = pd.read_csv(f'./power_data/solar_汙水廠_history(get_15m_power).csv', low_memory=False)
        d = pd.concat([old, data], axis=0, ignore_index=True)
        d = d.drop_duplicates(subset=['TIME_TO_INTERVAL'], keep='last')
        d.to_csv(f'./power_data/solar_汙水廠_history(get_15m_power).csv', index=None)
        print('okok')
        end_time = time.time()
        finish = end_time - start_time
        print(finish)
        time.sleep(3600-finish)
    else:
        m,s = result.strip().split(":")
        start_time = int(m)*60+int(s)
        time.sleep(3600-start_time)

2023-09-22 00:00:00
okok
4.0274977684021
2023-09-22 00:00:00
okok
3.8037540912628174
2023-09-22 00:00:00
okok
3.9220197200775146
2023-09-22 00:00:00
okok
3.756737470626831
2023-09-22 00:00:00
okok
3.9151089191436768
2023-09-22 00:00:00
okok
3.7980358600616455
2023-09-22 00:00:00
okok
3.906984806060791
2023-09-23 00:00:00
okok
3.0855965614318848
2023-09-23 00:00:00
okok
4.084815502166748
2023-09-23 00:00:00
okok
4.849104404449463
2023-09-23 00:00:00
okok
4.773914813995361
2023-09-23 00:00:00
okok
4.868921518325806
2023-09-23 00:00:00
okok
4.852119445800781
2023-09-23 00:00:00
okok
4.756804466247559
2023-09-23 00:00:00
okok
5.095213413238525
2023-09-23 00:00:00
okok
4.555338621139526
2023-09-23 00:00:00
okok
4.523196220397949
2023-09-23 00:00:00
okok
5.365859270095825
2023-09-23 00:00:00
okok
3.4598333835601807
EMAIL SEND OK
2023-09-23 00:00:00
okok
3.5477120876312256
2023-09-23 00:00:00
okok
3.5784475803375244
2023-09-23 00:00:00
okok
5.8247761726379395
2023-09-23 00:00:00
okok
6.837637

# 每15分爬取一次

In [ ]:
# while(True):
#     localtime = time.localtime()
#     result = time.strftime("%M:%S", localtime)
#     if(result<='02:00'):
#         start_time = time.time()
#         # 設定要產生的開始與結束日期
#         day = datetime.datetime.today()
#         day = pd.to_datetime(day, format='%Y%m%d')
#         start = pd.to_datetime(str(day.year)+'-'+str(day.month)+'-'+str(day.day))-datetime.timedelta(days=1)
#         print(start)
#         data = get_power_date(start)
#         old = pd.read_csv(f'./power_data/solar_汙水廠_history.csv', low_memory=False)
#         d = pd.concat([old, data], axis=0, ignore_index=True)
#         d = d.drop_duplicates(subset=['TIME_TO_INTERVAL'], keep='last')
#         d.to_csv(f'./power_data/solar_汙水廠_history.csv', index=None)
#         print('okok')
#         end_time = time.time()
#         finish = end_time - start_time
#         print(finish)
#         time.sleep(3600-finish)
#     else:
#         m,s = result.strip().split(":")
#         start_time = int(m)*60+int(s)
#         time.sleep(3600-start_time)

In [ ]:
# start = '2022-01-01'
# data = get_power_date(start)
# old = pd.read_csv(f'./power_data/solar_汙水廠_history.csv')
# d = pd.concat([old, data], axis=0, ignore_index=True)
# d = d.drop_duplicates(subset=['TIME_TO_INTERVAL'], keep='last')
# d.to_csv(f'./power_data/solar_汙水廠_history.csv', index=None)

In [ ]:
# merge_power_data = pd.read_csv('./power_data/merge_power_data(hour).csv')
# merge_power_data = merge_power_data[['kP','TIME_TO_INTERVAL']]
# merge_power_data.set_axis(['Power','TIME_TO_INTERVAL'], axis='columns', inplace=True)
# merge_power_data

In [ ]:
# solar_allhour = pd.read_csv('./power_data/solar_汙水廠(allhour).csv')
# solar_allhour = solar_allhour[['Power','TIME_TO_INTERVAL']]
# solar_allhour

In [ ]:
# merge_data = pd.concat([merge_power_data,solar_allhour],join='outer')
# merge_data = merge_data.drop_duplicates(keep='first',inplace=False)
# merge_data = merge_data.sort_values(by=['TIME_TO_INTERVAL'])
# merge_data = merge_data.reset_index(drop=True,inplace=False)
# merge_data.to_csv('./power_data/solar_汙水廠_history.csv',index=False)

In [ ]:
# merge_data

## 合併歷史原始資料成15分鐘一筆

In [ ]:
# MG1 = pd.read_csv(f'./power_data/original/MG1_PV_0.csv')
# MG1

In [ ]:
# import pandas as pd

# # files = [f'./power_data/original/MG1_PV_0.csv', f'./power_data/original/MG1_PV_500000.csv']
# # dataframes = []

# # for file in files:
# #     df = pd.read_csv(file)
# #     dataframes.append(df)
    
# # dataframes

# # 讀取資料
# data = pd.read_csv(f'./power_data/original/MG1_PV_8000000.csv', index_col=0)

# data['Time'] = pd.to_datetime(data['Time'])

# # 設定時間欄位為索引
# data.set_index('Time', inplace=True)

# # 將時間戳記轉換成 datetime 格式，並設定為 DataFrame 的 index

# # 將資料轉換成每 15 分鐘的平均值
# resampled_data = data.resample('15T').mean()
# resampled_data = resampled_data.reset_index()

# # 輸出轉換後的資料
# print(resampled_data)

# data_old = pd.read_csv(f'./power_data/original/2023_5_9轉15分/15m.csv')
# d = pd.concat([data_old, resampled_data], axis=0, ignore_index=True)
# d = d.drop_duplicates(subset=['Time'], keep='last')
# d.to_csv(f'./power_data/original/2023_5_9轉15分/15m.csv', index=None)
# # resampled_data.to_csv('./power_data/original/2023_5_9轉15分/15m.csv', index=None)